In [ ]:
import cv2
import numpy as np
import time
import math
import copy

WIND_X = 1000
WIND_Y = 800

axes = {"x" : 0, "y" : 1, "z" : 2}

markup_color = (250, 250, 250)

class Vector:
    def __init__ (self, coords_ = []):
        self.coords = coords_
    
    def get_coords (self):
        return self.coords
    
    def dotproduct (self, v):
        return sum ((a * b) for a, b in zip (self.coords, v.coords))

    def length (self):
        return math.sqrt (self.dotproduct (self))

    def subtr (self, v):
        return Vector ([a - b for a, b in zip (self.coords, v.coords)])

    def add (self, v):
        return Vector ([a + b for a, b in zip (self.coords, v.coords)])

    def mul (self, coeff):
        return Vector ([a * coeff for a in self.coords])

    def cos (self, v):
        return self.dotproduct (v) / (self.length () * v.length () + 0.0001)
    
    def change_coord (self, coord, val = 0, increment = False, invert = False):
        if (invert == True):
            self.coords [axes [coord]] *= -1
        
        else:
            if (increment == True):
                self.coords [axes [coord]] += val

            else:
                self.coords [axes [coord]] = val
    
    def rotate_2d (self, axis1, axis2, angle):
        orig1 = self.coords [axes [axis1]]
        orig2 = self.coords [axes [axis2]]
        
        rot1 =   orig1 * math.cos (angle) + orig2 * math.sin (angle)
        rot2 = - orig1 * math.sin (angle) + orig2 * math.cos (angle)

        self.coords [axes [axis1]] = rot1
        self.coords [axes [axis2]] = rot2
        
    def copy (self):
        return Vector (copy.deepcopy (self.coords))
    
    def scale (self, coeff):
        for i in range (len (self.coords)):
            self.coords [i] *= coeff

class Object:
    def __init__ (self, coords_):
        self.coords = coords_
        
    def draw (self, canvas):
        pass
    
    def get_coords (self):
        return self.coords.get_coords ()
    
    def get_coords_vec (self):
        return self.coords

class Light_emitter (Object):
    def __init__ (self, coords_, color_ = (255, 255, 255)):
        Object.__init__ (self, coords_)
        
        self.color = color_
    
    def get_color (self):
        return self.color
    
    def draw (self, canvas):
        canvas.draw_3d_circle (self.coords, 10, markup_color)

class Surface (Object):
    def __init__ (self, coords_):
        Object.__init__ (self, coords_)
    
    def draw (self, canvas):
        self.render (canvas)
    
    def render (self, canvas):
        pass
    
    def iterate_elements (self):
        pass
    
    def calc_element_lightening (self):
        pass
    
    def _norm_3_points (self, p1, p2, p3):
        u = p2.subtr (p1).get_coords ()
        v = p3.subtr (p1).get_coords ()
        
        n = Vector ([u [1] * v [2] - u [2] * v [1],
                     u [2] * v [0] - u [0] * v [2],
                     u [0] * v [1] - u [1] * v [0]])
        
        return n

class Triangle(Surface):
    def __init__ (self, p1_, p2_, p3_, color_ = (100, 100, 255)):
        Object.__init__ (self, p1_)
        
        self.p1 = p1_
        self.p2 = p2_
        self.p3 = p3_
        
        self.color = color_
    
    def draw (self, canvas, emitter, shift = [0, 0, 0]):
        self.render (canvas, emitter, shift)
    
    def render (self, canvas, emitter, shift = Vector ([0, 0, 0])):
        n = self._norm_3_points (self.p1, self.p2, self.p3)
        
        tr_color = self.calc_lightening (n, emitter)
                
        canvas.draw_3d_triangle (self.p1.add (shift),
                                 self.p2.add (shift),
                                 self.p3.add (shift),
                                 tr_color)
    
    def calc_lightening (self, n, emitter):
        light_vec = emitter.coords.subtr (self.coords)
        
        coeff = max (n.cos (light_vec), 0)
        
        result = [int (channel * coeff) for channel in self.color]
        
        return result
    
    def get_color (self):
        return self.color
    
    def change_coord (self, coord, val = 0, increment = False):
        for p in [self.p1, self.p2, self.p3]:
            p.change_coord (coord, val, increment)
    
    def scale (self, coeff):
        for p in [self.p1, self.p2, self.p3]:
            p.scale (coeff)

class Triangle_mesh (Surface):
    def __init__ (self, coords_, effective_spatial_dimension_ = 0, triangles_ = []):
        Surface.__init__ (self, coords_)
        
        self.triangles = triangles_
        
        self.effective_spatial_dimension = effective_spatial_dimension_
        
    def generate_triangulation (self):
        pass
    
    def get_effective_spatial_dimension (self):
        return self.effective_spatial_dimension

    def draw (self, canvas, emitter):
        for tr in sorted (self.triangles, key = lambda tr: -(tr.p1.get_coords () [2] +
                                                             tr.p2.get_coords () [2] +
                                                             tr.p3.get_coords () [2])):
            tr.draw (canvas, emitter, self.coords)
        
    def rotate (self, axis, step, increment = True):
        ind1, ind2 = [ax for ax in axes if ax not in [axis]] [:]
        
        for tr in self.triangles:
            for p in [tr.p1, tr.p2, tr.p3]:
                p.rotate_2d (ind1, ind2, step)
    
    def get_triangles (self):
        return self.triangles
    
    def change_coord (self, coord, val = 0, increment = False):
        self.coords.change_coord (coord, val, increment)

    def change_coord_mesh (self, coord, val = 0, increment = False):
        for tr in self.triangles:
            tr.change_coord (coord, val, increment)
        
    def scale (self, coeff):
        for tr in self.triangles:
            tr.scale (coeff)
    
class Radially_symmetrical_tri (Triangle_mesh):
    def __init__ (self, coords_, r_, h_, color_, stripes_num_, triangles_ = 3):
        Triangle_mesh.__init__ (self, coords_, r_)
        
        self.r     = r_
        self.h     = h_
        self.color = color_
        
        self.stripes_num = stripes_num_
        self.generate_triangulation (triangles_)
    
    def get_stripe_rad (self, r, i, h_step):
        stripe_rad_curr = r
        stripe_rad_next = r
        
        return stripe_rad_curr, stripe_rad_next
    
    def generate_triangulation (self, triangles_):
        h_step = self.h / self.stripes_num
        angle_step = 2 * math.pi / self.stripes_num
        
        for i in range (self.stripes_num):
            for j in range (self.stripes_num):
                stripe_rad_curr, stripe_rad_next = self.get_stripe_rad (self.r, i, h_step)
                
                p1 = Vector ([stripe_rad_curr * math.sin (j * angle_step),
                      stripe_rad_curr * math.cos (j * angle_step),
                      - self.r + i * h_step])
                
                p2 = Vector ([stripe_rad_curr * math.sin ((j + 1) * angle_step),
                      stripe_rad_curr * math.cos ((j + 1) * angle_step),
                      - self.r + i * h_step])
                
                p3 = Vector ([stripe_rad_next * math.sin (j * angle_step),
                      stripe_rad_next * math.cos (j * angle_step),
                      - self.r + (i + 1) * h_step])
                
                p4 = Vector ([stripe_rad_next * math.sin ((j + 1) * angle_step),
                      stripe_rad_next * math.cos ((j + 1) * angle_step),
                      - self.r + (i + 1) * h_step])
                
                p5 = p2.copy ()
                p6 = p3.copy ()
                
                new_triangle_1 = Triangle (p1, p2, p3, color_ = self.color)
                new_triangle_2 = Triangle (p4, p6, p5, color_ = self.color)

                if (triangles_ == 1 or triangles_ == 3):
                    self.triangles.append (new_triangle_1)
                
                if (triangles_ == 2 or triangles_ == 3):
                    self.triangles.append (new_triangle_2)

class Sphere_tri (Radially_symmetrical_tri):
    def __init__ (self, coords_, r_, h_, color_, stripes_num_, triangles_ = 3):
        Radially_symmetrical_tri.__init__ (self, coords_, r_, 2 * r_, color_, stripes_num_,
                                          triangles_)
    
    def get_stripe_rad (self, r, i, h_step):
        stripe_rad_curr = math.sqrt (self.r**2 - \
                    (- self.r + i * h_step)**2)
                
        stripe_rad_next = math.sqrt (self.r**2 - \
            (- self.r + (i + 1) * h_step)**2)

        return stripe_rad_curr, stripe_rad_next

class Tree_tri (Triangle_mesh):
    def __init__ (self, coords_, length_, color_, stripes_num_, max_depth_,
                  angle_range_, length_decrement_factor_, branching_factor_, triangles_ = 3):
        Triangle_mesh.__init__ (self, coords_, length_)
        
        self.length = length_
        self.r      = length_ / 19
        self.color  = color_
        
        self.stripes_num = stripes_num_
        
        self.angle_range             = angle_range_
        self.length_decrement_factor = length_decrement_factor_
        self.max_depth               = max_depth_
        self.branching_factor        = branching_factor_
        
        self.generate_triangulation (triangles_)
    
    def construct_tree (self, base_element, max_depth, length):
        resultant_triangles = []
        
        if (max_depth > 0):
            new_base = copy.deepcopy (base_element)
            
            resultant_triangles += base_element.get_triangles ()
            new_base.scale (self.length_decrement_factor)
            
            #angle_step = self.angle_range / (self.branching_factor - 1)
            
            for i in range (self.branching_factor):
                (_, child_triangles) = self.construct_tree (new_base, max_depth - 1,
                    length * self.length_decrement_factor)
                
                child_triangle_mesh = Triangle_mesh (Vector ([0, 0, 0]), triangles_ = child_triangles)
                
                child_triangle_mesh.rotate ("x", self.angle_range / 2)
                child_triangle_mesh.rotate ("z", 2 * math.pi / self.branching_factor * i)

                child_triangle_mesh.change_coord_mesh ("z", length * 0.93, increment = True)

                resultant_triangles += child_triangle_mesh.get_triangles ()
                
        return (Triangle_mesh (Vector ([0, 0, 0]), triangles_ = copy.deepcopy (resultant_triangles)),
            copy.deepcopy (resultant_triangles))
    
    def generate_triangulation (self, triangles_):
        base_element = Radially_symmetrical_tri (self.coords, self.r, self.length,
                                                 self.color, self.stripes_num, triangles_ = triangles_)
        
        (_, tri) = self.construct_tree (base_element, self.max_depth, self.length)
        
        self.triangles = tri

class Cube (Triangle_mesh):
    def __init__ (self, coords_, sz_, color_):
        Triangle_mesh.__init__ (self, coords_, sz_)
        
        self.sz = sz_
        self.color = color_
        
        self.generate_triangulation ()
    
    def generate_triangulation (self):
        d = copy.deepcopy
        
        p000 = Vector ([0, 0, 0])
        p001 = Vector ([0, 0, self.sz])
        p010 = Vector ([0, self.sz, 0])
        p011 = Vector ([0, self.sz, self.sz])
        p100 = Vector ([self.sz, 0, 0])
        p101 = Vector ([self.sz, 0, self.sz])
        p110 = Vector ([self.sz, self.sz, 0])
        p111 = Vector ([self.sz, self.sz, self.sz])

        self.triangles.append (Triangle (d (p001), d (p000), d (p011), self.color))
        self.triangles.append (Triangle (d (p011), d (p000), d (p010), self.color))
        self.triangles.append (Triangle (d (p100), d (p101), d (p111), self.color))
        self.triangles.append (Triangle (d (p110), d (p100), d (p111), self.color))
        
        self.triangles.append (Triangle (d (p000), d (p001), d (p100), self.color))
        self.triangles.append (Triangle (d (p001), d (p101), d (p100), self.color))
        self.triangles.append (Triangle (d (p010), d (p110), d (p011), self.color))
        self.triangles.append (Triangle (d (p011), d (p110), d (p111), self.color))

        self.triangles.append (Triangle (d (p010), d (p000), d (p110), self.color))
        self.triangles.append (Triangle (d (p110), d (p000), d (p100), self.color))
        self.triangles.append (Triangle (d (p001), d (p011), d (p111), self.color))
        self.triangles.append (Triangle (d (p101), d (p001), d (p111), self.color))

class Menger_sponge (Triangle_mesh):
    def __init__ (self, coords_, sz_, color_, depth_):
        Triangle_mesh.__init__ (self, coords_, sz_)
        
        self.color = color_
        self.sz = sz_
        self.depth = depth_
        self.generate_triangulation ()
        
    def generate_triangulation (self):
        base_element = Cube (self.coords, self.sz, self.color)
        
        element_resized  = Triangle_mesh (Vector ([0, 0, 0]),
            triangles_ = copy.deepcopy (base_element.get_triangles ()))
        
        #element_resized = base_element
        
        if (self.depth == 0):
            self.triangles = base_element.get_triangles ()
            return
        
        s = 1.0 / 3 #scale
        sponge_triangles = []
        
        for _ in range (self.depth):
            #print (element_resized)
            
            sponge_triangles = []
            element_resized.scale (s)
            
            #print ("len", len (sponge_triangles))
            
            for i in range (3):
                for j in range (3):
                    for k in range (3):
                        #print (i * s * self.sz)
                        
                        if ((i == 1 and j == 1) or (i == 1 and k == 1) or (j == 1 and k == 1)):
                            continue
                        
                        element_resized.change_coord_mesh ("x", i * s * self.sz, increment = True)
                        element_resized.change_coord_mesh ("y", j * s * self.sz, increment = True)
                        element_resized.change_coord_mesh ("z", k * s * self.sz, increment = True)
                        
                        tmp = copy.deepcopy (element_resized.get_triangles ())
                        #print (len (tmp), "fu")
                        sponge_triangles += tmp

                        element_resized.change_coord_mesh ("x", -i * s * self.sz, increment = True)
                        element_resized.change_coord_mesh ("y", -j * s * self.sz, increment = True)
                        element_resized.change_coord_mesh ("z", -k * s * self.sz, increment = True)

                        #print (i, j, k)
            
            element_resized = Triangle_mesh (self.coords, triangles_ = sponge_triangles)
        
        #print (len (sponge_triangles), "leen", sponge_triangles [10].p1.get_coords ())
        self.triangles = sponge_triangles #element_resized.get_triangles ()
        #print (self.get_coords (), "coords")

class Canvas:
    def __init__ (self, xsz_, ysz_, zsz_, centerx_, centery_):
        self.xsz = xsz_
        self.ysz = ysz_
        self.zsz = zsz_
        self.centerx = centerx_
        self.centery = centery_
        
        self.canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55
        
    def get_canvas (self):
        return self.canvas
    
    def refresh (self):
        self.canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55
        self.draw_space_box ()
    
    def _transform_point (self, p):
        coords = p.get_coords ()
        
        x = int ((coords [0] / (coords [2] + 0) + self.centerx) * WIND_X / self.xsz)
        y = int ((coords [1] / (coords [2] + 0) + self.centery) * WIND_Y / self.ysz)
        
        return x, y
    
    def draw_3d_line (self, p1, p2, color, thickness = 1):
        x1, y1 = self._transform_point (p1)
        x2, y2 = self._transform_point (p2)
        
        cv2.line (self.canvas, (x1, y1), (x2, y2), color, thickness)
    
    def draw_3d_triangle (self, p1, p2, p3, color):
        x1, y1 = self._transform_point (p1)
        x2, y2 = self._transform_point (p2)
        x3, y3 = self._transform_point (p3)
        
        contour = np.array ([(x1, y1), (x2, y2), (x3, y3)])
        
        cv2.drawContours (self.canvas, [contour], 0, color, -1)
        
    def draw_3d_circle (self, p, r, color):
        x, y = self._transform_point (p)
        
        cv2.circle (self.canvas, (x, y), int (r / p.get_coords () [2]), color)
    
    def draw_space_box (self):
        lucc = Vector ([- self.centerx, - self.centery, 1]) #left-upper-close corner
        ludc = Vector ([- self.centerx, - self.centery, self.zsz])
        ldcc = Vector ([- self.centerx, self.ysz - self.centery, 1])
        lddc = Vector ([- self.centerx, self.ysz - self.centery, self.zsz])

        rucc = Vector ([self.xsz - self.centerx, - self.centery, 1])
        rudc = Vector ([self.xsz - self.centerx, - self.centery, self.zsz])
        rdcc = Vector ([self.xsz - self.centerx, self.ysz - self.centery, 1])
        rddc = Vector ([self.xsz - self.centerx, self.ysz - self.centery, self.zsz])
        
        self.draw_3d_line (lucc, ludc, markup_color)
        self.draw_3d_line (ldcc, lddc, markup_color)
        self.draw_3d_line (rucc, rudc, markup_color)
        self.draw_3d_line (rdcc, rddc, markup_color)

        self.draw_3d_line (ludc, rudc, markup_color)
        self.draw_3d_line (rudc, rddc, markup_color)
        self.draw_3d_line (rddc, lddc, markup_color)
        self.draw_3d_line (lddc, ludc, markup_color)
        
    def put_text (self, text, x, y, color = (100, 250, 130)):
        cv2.putText (self.canvas, text, (x, y),
            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
    
    def is_in (self, point, shift = 0):
        is_in_box = True
        
        coords = point.get_coords ()
        
        coll_axes = {"x" : 0, "y" : 0, "z" : 0}
        
        #return is_in_box, coll_axes
        
        if (coords [0] <          - self.centerx + shift or
            coords [0] > self.xsz - self.centerx - shift):
            is_in_box = False
            coll_axes ["x"] = 1

        if (coords [1] <          - self.centery + shift or
            coords [1] > self.ysz - self.centery - shift):
            is_in_box = False
            coll_axes ["y"] = 1

        if (coords [2] < 1 + shift or
            coords [2] > 1 + self.zsz - shift):
            is_in_box = False
            coll_axes ["z"] = 1
        
        return is_in_box, coll_axes

class Field:
    def __init__ (self):
        pass
    
    def calc_acceleration (self, coords):
        return Vector ([0.0, 0.0, 0.0])

class Isotrophiс_field (Field):
    def __init__ (self, acceleration_vector_):
        Field.__init__ (self)
        self.acceleration_vector = acceleration_vector_
    
    def calc_acceleration (self, coords):
        return self.acceleration_vector

class Object_manager:
    def __init__ (self, canvas_):
        self.emitter = Light_emitter (Vector ([0.0, 0, 0.5]))
        self.canvas = canvas_
        
        #self.canvas = Canvas (4, 3.2, 6, 2, 1.6)

        self.objects = []
        
        self.to_refresh = True
        
        self.light_step = 0.2
        self.rot_step   = 0.1
        
        self.explicit_fields = []
        
    def add_object (self, obj):
        self.objects.append (obj)
    
    def remove (self, obj):
        self.objects.remove (obj)
        print ("obj num" , len (self.objects))
    
    def pop (self, ind):
        return self.objects.pop (ind)

    def add_objects (self, objects):
        self.objects += objects
    
    def add_explicit_field (self, field):
        self.explicit_fields.append (field)
    
    def draw (self):
        if (self.to_refresh == True):
            #self.to_refresh = False
            self.canvas.refresh ()

            for obj in sorted (self.objects, key = lambda obj: -obj.get_coords () [2]):
                obj.draw (self.canvas, self.emitter)

            self.emitter.draw (self.canvas)
    
    def rotate (self, axis, angle):
        for obj in self.objects:
            obj.rotate (axis, angle)
    
    def get_canvas (self):
        return self.canvas.get_canvas ()
    
    def handle_keyboard (self, key):
        for obj in self.objects:
            obj.handle_keyboard (key)
    
    def move (self, canvas):
        for obj in self.objects:
            obj.move (canvas, self.explicit_fields)
            
    def handle_collisions (self, distance_lower_limit):
        interacting_objects_inds = []
        
        for i in range (len (self.objects [1 : ])):
            obj = self.objects [i + 1]
            
            distance = obj.coords.subtr (self.objects [0].coords).length ()
            
            if (distance < distance_lower_limit):
                interacting_objects_inds.append (i + 1)
                
        return interacting_objects_inds

class Target (Object):
    def __init__ (self, coords_, velocity_, triangle_mesh_, points_ = 3,
                  ignore_fields_ = False):
        Object.__init__ (self, triangle_mesh_.coords)
        
        self.velocity = velocity_
        self.points = points_
        
        self.triangle_mesh = triangle_mesh_
        self.ignore_fields = ignore_fields_
    
    def move (self, canvas, fields):
        effective_acceleration = Vector ([0, 0, 0])

        if (self.ignore_fields == False):
            for field in fields:
                effective_acceleration = effective_acceleration.add \
                    (field.calc_acceleration (self.coords))
        
        is_in, collision_axes = canvas.is_in (self.coords,
        self.triangle_mesh.get_effective_spatial_dimension ())

        if (is_in == False):
            for axis in collision_axes.keys ():
                if (collision_axes [axis] == 1):
                    self.velocity.change_coord (axis, invert = True)
                    self.velocity.change_coord (axis,
                        -effective_acceleration.get_coords ()
                        [axes [axis]], increment = True)
        
        self.velocity = self.velocity.add (effective_acceleration)
        self.triangle_mesh.coords = self.triangle_mesh.coords.add (self.velocity)
        self.coords = self.triangle_mesh.coords
                
    def draw (self, canvas, emitter):
        self.triangle_mesh.draw (canvas, emitter)
#         canvas.draw_3d_line (sun.get_coords_vec (),
#                              self.triangle_mesh.get_coords_vec (),
#                              self.triangle_mesh.triangles [0].get_color ())
    
    def get_effective_spatial_dimension (self):
        return self.triangle_mesh.get_effective_spatial_dimension ()
    
    def change_velocity (self, coord, val = 0, increment = False, invert = False):
        self.velocity.change_coord (coord, val, increment, invert)
    
    def handle_keyboard (self, key):
        pass

class Player (Object):
    def __init__ (self, coords_, triangle_mesh_, name_ = "noname"):
        Object.__init__ (self, coords_)
        
        self.triangle_mesh = triangle_mesh_
        
    def handle_keyboard (self, key):
        if (key == ord ("i")):
            self.triangle_mesh.change_coord ("y", -0.1, increment = True)
        
        if (key == ord ("k")):
            self.triangle_mesh.change_coord ("y", 0.1, increment = True)
    
        if (key == ord ("j")):
            self.triangle_mesh.change_coord ("x", -0.1, increment = True)
        
        if (key == ord ("l")):
            self.triangle_mesh.change_coord ("x", 0.1, increment = True)
    
    def draw (self, canvas, emitter):
        self.triangle_mesh.draw (canvas, emitter)
    
    def move (self, canvas, fields):
        pass
    
class Game:
    def __init__ (self, scenario_, canvas_):
        self.scenario = scenario_
        self.canvas   = canvas_
        
        self.starting_time = time.time ()
        self.last_adding_time = self.starting_time
        
        self.player = Player (Vector ([0, 0, 1.4]), Menger_sponge
                              (Vector ([0.1, 0.1, 1]), sz_ = 0.3,
                               color_ = (100, 20, 230), depth_ = 1))
    
        self.object_manager = Object_manager (canvas_)
        
        self.object_manager.add_object (self.player)
        
        grav_field = Isotrophiс_field (Vector ([-0.01, 0, 0]))
        self.object_manager.add_explicit_field (grav_field)
                
        self.status = "playing"
        self.score = 0
    
    def add_random_object (self):
        def rv (limit, multiplier = 0.01): #random velocity
            rand_val = np.random.randint (limit)
            vel = (float (rand_val) - limit / 2.0) * multiplier
            
            return vel
        
        new_object =  Target (Vector ([0, 0, 0]),
                              
                              #Vector ([-0.006, 0.007, 0.05]),
                              Vector ([rv (15), rv (20), rv (35)]),
                              
                              #Sphere_tri (Vector ([0, 0, 4]),
                              #0.2, 0.2, (100, 200, 200), 5)
                              Triangle_mesh (Vector ([0, 0, 4])))
        
        self.add_object (new_object)
    
    def add_object (self, new_object):
        self.object_manager.add_object (new_object)
    
    def draw (self, canvas):
        self.object_manager.draw ()
        
        self.canvas.put_text ("Score: " + str (self.score), 30, 30)
        self.canvas.put_text ("Time: " + str (self.scenario ["duration"] +
            self.starting_time - time.time ()) [:5], 30, 80)
    
    def handle_collisions (self):
        interacting_targets_inds = self.object_manager.handle_collisions\
            (self.scenario ["eating distance"])
        
        for i in interacting_targets_inds:
            obj = self.object_manager.pop (i)
            
            self.score += obj.points

            del obj
    
    def move (self):
        self.object_manager.move (self.canvas)
    
    def handle_keyboard (self, key):
        self.object_manager.handle_keyboard (key)
        
    def add_new (self):
        time_since_last_adding = - self.last_adding_time + time.time ()
        
        if (time_since_last_adding > self.scenario ["target spawn delay"]):
            self.add_random_object ()
            self.last_adding_time = time.time ()
    
    def remove_escaped (self):
        pass
        #to_remove = object_manager.
    
    def game_iteration (self):
        self.add_new ()
        self.remove_escaped ()
        
        self.move ()
        self.handle_collisions ()
        
        if (self.score < 0):
            self.status = "lose"
        
        if (time.time () - self.starting_time > self.scenario ["duration"]):
            self.status = "win"

class Game_manager:
    def __init__ (self, config_):
        self.config = config_
        self.canvas = Canvas (2, 1.6, 5, 1, 0.8)
        
        self.game = Game (self.config, self.canvas)
        self.status = "waiting"
        
    def handle_keyboard (self):
        key = cv2.waitKey (1) & 0xFF
        
        if (key == ord ('q')):
            return {"exit" : True}
        
        if (self.status == "waiting"):
            if (key == ord ("n")):
                self.status = "playing"
                del self.game
                
                self.game = Game (self.config, self.canvas)

        if (self.status == "playing"):
            self.game.handle_keyboard (key)
        
        if (self.status == "finished"):
            if (key == ord ("c")):
                self.status = "waiting"
    
        return {"exit" : False}
            
    def draw (self):
        if (self.status == "waiting"):
            self.canvas.put_text ("Press n to play, ijkl - control", 30, 50)
        
        if (self.status == "playing"):
            self.game.draw (self.canvas)
        
        if (self.status == "finished"):
            self.canvas.put_text (self.game.status + " with " + str (self.game.score) +
                "points, press c", 30, 50)
    
    def on_idle (self):
        if (self.game.status in ["win", "lose"]):
            self.status = "finished"
            self.game.status += " "
        
        if (self.status == "playing"):
            self.game.game_iteration ()
    
    def get_canvas (self):
        return self.canvas.get_canvas ()

game_config = {"duration"           : 155,
               "target spawn delay" : 3,
               "eating distance"    : 0.5}

game_manager = Game_manager (game_config)

while (True):
    if (game_manager.handle_keyboard () ["exit"] == True):
        break
    
    game_manager.on_idle ()
    game_manager.draw ()
    
    cv2.imshow ("render", game_manager.get_canvas ())

cv2.waitKey (0)
cv2.destroyAllWindows ()